In [2]:
import numpy as np
import math
import heapq

class Node:
    """
    A node class for A* Pathfinding.
    """
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

    def __lt__(self, other):
        return self.f < other.f  # For heap comparison based on f-value


def return_path(current_node, maze):
    """
    Returns the path from start to end by backtracking parents.
    """
    path = []
    no_rows, no_columns = np.shape(maze)
    result = [[-1 for _ in range(no_columns)] for _ in range(no_rows)]
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    path = path[::-1]  # Reverse to start-to-end order
    for i, position in enumerate(path):
        result[position[0]][position[1]] = i
    return result


def search(maze, cost, start, end):
    """
    A* search using a Min-Heap for priority queue.
    """
    start_node = Node(None, tuple(start))
    end_node = Node(None, tuple(end))
    start_node.g = start_node.h = start_node.f = 0
    end_node.g = end_node.h = end_node.f = 0

    # Min-Heap for open list
    open_list = []
    closed_list = set()

    # Add the start node to the heap
    heapq.heappush(open_list, start_node)

    move = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # Up, Left, Down, Right
    no_rows, no_columns = np.shape(maze)

    while open_list:
        # Pop the node with the lowest f-value
        current_node = heapq.heappop(open_list)
        closed_list.add(current_node.position)

        # If goal is reached, return the path
        if current_node == end_node:
            return return_path(current_node, maze)

        # Generate children
        for new_position in move:
            node_position = (current_node.position[0] + new_position[0],
                             current_node.position[1] + new_position[1])

            # Check if within bounds
            if (node_position[0] < 0 or node_position[0] >= no_rows or
                node_position[1] < 0 or node_position[1] >= no_columns):
                continue

            # Check if walkable
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            child = Node(current_node, node_position)

            # If already evaluated, skip
            if child.position in closed_list:
                continue

            # Calculate g, h, and f values
            child.g = current_node.g + cost
            child.h = math.sqrt((child.position[0] - end_node.position[0]) ** 2 +
                                (child.position[1] - end_node.position[1]) ** 2)  # Euclidean
            child.f = child.g + child.h

            # If already in open list with lower g, skip
            if any(open_node for open_node in open_list if child == open_node and child.g >= open_node.g):
                continue

            # Add the child to the heap
            heapq.heappush(open_list, child)


if __name__ == '__main__':
    maze = [[0, 1, 0, 0, 0, 0],
            [0, 0, 0, 1, 1, 0],
            [0, 1, 1, 1, 0, 0],
            [0, 1, 0, 0, 0, 1],
            [0, 0, 0, 1, 0, 0]]

    start = [0, 0]  # Starting position
    end = [4, 5]    # Ending position
    cost = 1        # Cost per movement

    path = search(maze, cost, start, end)


## Final result path

In [16]:
print('\n'.join([''.join(["{:" ">3d}".format(item) for item in row])
      for row in path]))

  0 -1 -1 -1 -1 -1
  1 -1 -1 -1 -1 -1
  2 -1 -1 -1 -1 -1
  3 -1  7  8  9 -1
  4  5  6 -1 10 11
